# AIS Data Cleaning

In [1]:
import aisdb
from aisdb import DBConn, DBQuery, DomainFromPoints
from datetime import datetime, timedelta
import nest_asyncio
nest_asyncio.apply()

dbpath='YOUR_DATABASE.db' # Define the path to your database

## Query data without cleaning

In [2]:
# Set the start and end times for the query
start_time = datetime.strptime("2018-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime("2018-01-02 00:00:00", '%Y-%m-%d %H:%M:%S')

# A circle with a 100km radius around the location point
domain = DomainFromPoints(points=[(-63.6, 44.6)], radial_distances=[50000])

with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn,
        start=start_time,
        end=end_time,
        callback=aisdb.database.sqlfcn_callbacks.in_timerange_validmmsi,
    )
    rowgen = qry.gen_qry()
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    
    aisdb.web_interface.visualize(
        tracks,
        domain=domain,
        visualearth=True,
        open_browser=True,
    )

Serving HTTP assets on localhost:3000
Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage
127.0.0.1:39416 - received: {'msgtype': 'meta'}
127.0.0.1:39418 - received: {'msgtype': 'validrange'}
127.0.0.1:39418 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...


## After Cleaning

In [3]:
def color_tracks(tracks):
    ''' set the color of each vessel track using a color name or RGB value '''
    for track in tracks:
        track['color'] = 'yellow'
        yield track

In [4]:
maxdelta = timedelta(hours=24)  # the maximum time interval
distance_threshold = 20000      # the maximum allowed distance (meters) between consecutive AIS messages
speed_threshold = 50            # the maximum allowed vessel speed in consecutive AIS messages
minscore = 1e-6                 # the minimum score threshold for track segment validation

with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn,
        start=start_time,
        end=end_time,
        callback=aisdb.database.sqlfcn_callbacks.in_timerange_validmmsi,
    )
    rowgen = qry.gen_qry()
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    
    # Split the tracks into segments based on the maximum time interval
    track_segments = aisdb.split_timedelta(tracks, maxdelta)
    
    # Encode the track segments to clean and validate the track data
    tracks_encoded = aisdb.encode_greatcircledistance(track_segments, 
                                                      distance_threshold=distance_threshold, 
                                                      speed_threshold=speed_threshold, 
                                                      minscore=minscore)
    tracks_colored = color_tracks(tracks_encoded)
    
    aisdb.web_interface.visualize(
        tracks_colored,
        domain=domain,
        visualearth=True,
        open_browser=True,
    )

Serving HTTP assets on localhost:3000
Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage
127.0.0.1:56968 - received: {'msgtype': 'meta'}
127.0.0.1:56984 - received: {'msgtype': 'validrange'}
127.0.0.1:56984 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...
